<img src="assets/routing.png" align="left" width="600" style="margin-right:15px;"/>

In [ ]:
from dotenv import load_dotenv, find_dotenv
from env_utils import doublecheck_env
from langchain_anthropic import ChatAnthropic

path = find_dotenv()
print("Loaded env from:", path)

# Load environment variables from .env
load_dotenv(find_dotenv())

# Check and print results
doublecheck_env(path)

llm = ChatAnthropic(model="claude-sonnet-4-5-20250929")

In [ ]:
from typing_extensions import Literal
from langchain.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END
from IPython.display import display, Image
from pydantic import BaseModel, Field
from typing_extensions import TypedDict

# Schema for structured output to use as routing logic
class Route(BaseModel):
    step: Literal["performance_analysis", "security_analysis", "capacity_analysis"] = Field(
        None, description="The next step in the routing process"
    )


# Augment the LLM with schema for structured output
router = llm.with_structured_output(Route)


# State
class State(TypedDict):
    input: str
    decision: str
    output: str


# Nodes
def llm_call_1(state: State):
    """Generate Performance Analysis"""

    prompt = f"""As a BT Network Performance Analyst, provide a single concise paragraph (4-6 sentences) performance analysis for: {state['input']}

Include: key KPIs (throughput, latency, availability), current metrics vs SLA targets, one performance trend, and one optimization recommendation.

Keep it professional, data-driven, and concise in ONE paragraph only."""

    result = llm.invoke(prompt)
    return {"output": result.content}


def llm_call_2(state: State):
    """Generate Security Analysis"""

    prompt = f"""As a BT Network Security Analyst, provide a single concise paragraph (4-6 sentences) security analysis for: {state['input']}

Include: threat assessment, security posture evaluation, compliance status, risk rating, and one key security recommendation.

Keep it professional, risk-focused, and concise in ONE paragraph only."""

    result = llm.invoke(prompt)
    return {"output": result.content}


def llm_call_3(state: State):
    """Generate Capacity Analysis"""

    prompt = f"""As a BT Network Capacity Planner, provide a single concise paragraph (4-6 sentences) capacity analysis for: {state['input']}

Include: current utilization, growth projection, one key bottleneck, scaling requirements, and investment recommendation.

Keep it professional, forward-looking, and concise in ONE paragraph only."""

    result = llm.invoke(prompt)
    return {"output": result.content}


def llm_call_router(state: State):
    """Route the input to the appropriate analysis type"""

    # Run the augmented LLM with structured output to serve as routing logic
    decision = router.invoke(
        [
            SystemMessage(
                content="""Route the input to the appropriate network analysis type:
- performance_analysis: for queries about network performance, KPIs, latency, throughput, SLA
- security_analysis: for queries about security, threats, vulnerabilities, compliance, risk
- capacity_analysis: for queries about capacity, scaling, growth, bottlenecks, planning

Analyze the user's query and determine which type of analysis is most appropriate."""
            ),
            HumanMessage(content=state["input"]),
        ]
    )

    return {"decision": decision.step}


# Conditional edge function to route to the appropriate node
def route_decision(state: State):
    # Return the node name you want to visit next
    if state["decision"] == "performance_analysis":
        return "llm_call_1"
    elif state["decision"] == "security_analysis":
        return "llm_call_2"
    elif state["decision"] == "capacity_analysis":
        return "llm_call_3"


# Build workflow
router_builder = StateGraph(State)

# Add nodes
router_builder.add_node("llm_call_1", llm_call_1)
router_builder.add_node("llm_call_2", llm_call_2)
router_builder.add_node("llm_call_3", llm_call_3)
router_builder.add_node("llm_call_router", llm_call_router)

# Add edges to connect nodes
router_builder.add_edge(START, "llm_call_router")
router_builder.add_conditional_edges(
    "llm_call_router",
    route_decision,
    {  # Name returned by route_decision : Name of next node to visit
        "llm_call_1": "llm_call_1",
        "llm_call_2": "llm_call_2",
        "llm_call_3": "llm_call_3",
    },
)
router_builder.add_edge("llm_call_1", END)
router_builder.add_edge("llm_call_2", END)
router_builder.add_edge("llm_call_3", END)

# Compile workflow
router_workflow = router_builder.compile()

# Show the workflow
display(Image(router_workflow.get_graph().draw_mermaid_png()))

# Invoke
# state = router_workflow.invoke({"input": "Analyze the performance of our 5G Core Network in the London region"})
# print(state["output"])

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Clear cell output
clear_output(wait=True)

# Create all widgets fresh
topic_input = widgets.Text(
    value='Analyze the security posture of our 5G Core Network',
    placeholder='Enter your network query...',
    description='Query:',
    continuous_update=False,
    layout=widgets.Layout(width='500px')
)

submit_btn = widgets.Button(
    description='Generate Analysis',
    button_style='primary',
    icon='search'
)

result_html = widgets.HTML(value='')

def on_button_click(btn):
    """Handle button click"""
    query = topic_input.value.strip()
    
    if not query:
        result_html.value = "<p>⚠️ Please enter a network query!</p>"
        return
    
    # Show loading state
    btn.disabled = True
    result_html.value = f"<p>🔄 Processing query: {query}...</p>"
    
    try:
        # Invoke workflow
        state = router_workflow.invoke({"input": query})
        
        # Determine analysis type based on decision
        analysis_type_display = {
            "performance_analysis": "Performance Analysis",
            "security_analysis": "Security Analysis",
            "capacity_analysis": "Capacity Analysis"
        }.get(state.get('decision', ''), "Analysis")
        
        # Build HTML string with results
        html = f"""
        <div style="font-family: 'Segoe UI', Arial, sans-serif; padding: 15px; background: #f8f9fa; border-radius: 5px;">
            <h3 style="color: #0066cc; margin-top: 0;">BT Network Intelligence: {analysis_type_display}</h3>
            <hr style="border: 1px solid #0066cc;">
            
            <div style="margin: 20px 0;">
                <h4 style="color: #0066cc;">Query: {query}</h4>
                <div style="background: white; padding: 15px; border-left: 3px solid #0066cc; margin-top: 10px;">
                    <pre style="white-space: pre-wrap; margin: 0; font-family: 'Segoe UI', Arial, sans-serif;">{state['output']}</pre>
                </div>
            </div>
            
            <div style="margin-top: 20px; padding: 10px; background: #e8f4f8; border-radius: 3px;">
                <small><strong>Routing Decision:</strong> {state.get('decision', 'N/A')}</small>
            </div>
        </div>
        """
        result_html.value = html
        
    except Exception as e:
        result_html.value = f"<p>❌ Error: {str(e)}</p>"
    
    finally:
        btn.disabled = False

# Attach handler
submit_btn.on_click(on_button_click)

# Display
display(widgets.VBox([
    widgets.HTML("<h3>🌐 BT Network Intelligence Router</h3>"),
    widgets.HTML("""<p>Enter a network query and the system will intelligently route it to the appropriate analysis type:</p>
    <ul>
        <li><strong>Performance Analysis:</strong> for queries about KPIs, latency, throughput, SLA performance</li>
        <li><strong>Security Analysis:</strong> for queries about threats, vulnerabilities, compliance, security posture</li>
        <li><strong>Capacity Analysis:</strong> for queries about scaling, growth projections, bottlenecks, capacity planning</li>
    </ul>
    <p><em>Examples: "Analyze the security of our 5G Core", "What is the performance status of London region?", "Capacity planning for fiber network expansion"</em></p>"""),
    widgets.HBox([topic_input, submit_btn]),
    result_html
]))